In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
from pandas import json_normalize

# df_train = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_train.json') #Results contain the required data
df_train = pd.read_json('/content/drive/MyDrive/FinNUM_dataset/FinNum-3_ConCall_train.json')
df_train

,paragraph,target_num,category,offset_start,offset_end,claim
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0
1,Please note that some of the information you'l...,10.0,other,504,506,0
2,Please note that some of the information you'l...,8.0,other,536,537,0
3,Thank you Nancy and thanks to everyone for joi...,53.3,money,212,216,0
4,Thank you Nancy and thanks to everyone for joi...,3.0,date,243,244,0
...,...,...,...,...,...,...
8332,Our momentum in pharmacy care services was exc...,2018.0,date,56,60,0
8333,Our momentum in pharmacy care services was exc...,98.0,quantity_relative,102,104,0
8334,Our momentum in pharmacy care services was exc...,2020.0,date,282,286,0
8335,In care delivery our clinical leaders are appl...,99.0,quantity_relative,212,214,0


In [3]:
# df_test = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_train.json') #Results contain the required data
df_test = pd.read_json('/content/drive/MyDrive/FinNUM_dataset/FinNum-3_ConCall_train.json')
df_test

,paragraph,target_num,category,offset_start,offset_end,claim
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0
1,Please note that some of the information you'l...,10.0,other,504,506,0
2,Please note that some of the information you'l...,8.0,other,536,537,0
3,Thank you Nancy and thanks to everyone for joi...,53.3,money,212,216,0
4,Thank you Nancy and thanks to everyone for joi...,3.0,date,243,244,0
...,...,...,...,...,...,...
8332,Our momentum in pharmacy care services was exc...,2018.0,date,56,60,0
8333,Our momentum in pharmacy care services was exc...,98.0,quantity_relative,102,104,0
8334,Our momentum in pharmacy care services was exc...,2020.0,date,282,286,0
8335,In care delivery our clinical leaders are appl...,99.0,quantity_relative,212,214,0


In [7]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 3.3 MB 45.8 MB/s 
     |████████████████████████████████| 895 kB 52.5 MB/s 
     |████████████████████████████████| 61 kB 490 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

In [9]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook


## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [10]:
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = 2
config

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [11]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [12]:
SEED = 42

In [13]:
def reset_randomness(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # This implies torch.cuda.manual_seed_all(SEED) now
    # if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True  # About 15% slower but...
    torch.backends.cudnn.benchmark = False

In [ ]:
def prepare_features(seq_1, max_seq_length = 512, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [ ]:
prepare_features("Please note that some of the information youll hear during our discussion today will consist of forward-looking statements including without limitation those regarding revenue gross margin operating expenses other income and expense taxes capital allocation share repurchases dividends and future business outlook. Actual results or trends could differ materially from our forecast. For more information please refer to the risk factors discussed in Apple's most recently filed periodic reports on Form 10-K and Form 10-Q and the Form 8-K filed with the SEC today along with the associated press release.")


(tensor([[    0,  6715,  1591,    14,   103,     9,     5,   335,    47,   890,
           1798,   148,    84,  3221,   452,    40, 19438,     9,   556,    12,
           3690,  1997,   217,   396, 22830,   167,  2624,   903,  4200,  2759,
           1633,  4068,    97,  1425,     8,  5623,  2556,   812, 12278,   458,
           2851,  9222,  9354,  8599,     8,   499,   265,  3839,     4, 30144,
            775,    50,  3926,   115, 10356, 11463,    31,    84,  1914,     4,
            286,    55,   335,  2540,  9115,     7,     5,   810,  2433,  3373,
             11,  1257,    18,   144,   682,  1658, 27185,   690,    15,  8575,
            158,    12,   530,     8,  8575,   158,    12,  1864,     8,     5,
           8575,   290,    12,   530,  1658,    19,     5,  3614,   452,   552,
             19,     5,  3059,  1228,   800,     4,     2,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,   

In [ ]:
train_size = 0.8
train_data=df_train.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_data=df_train.drop(train_data.index).reset_index(drop=True)

In [ ]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        paragraph = self.data.paragraph[index]
        claim = self.data.claim[index]
        X, _  = prepare_features(paragraph)
        y = self.data.claim[index]
        return X, y
    
    def __len__(self):
        return self.len

In [ ]:
training_set = Intents(train_data)
print(training_set.__getitem__(0)[0].shape)

torch.Size([1, 512])


In [ ]:
training_set.__getitem__(0)

(tensor([[    0,  2709,     5,    76,    52,  1057,    14,    84,  1633,  2759,
             40,    28,  5700,  3269,    19,    99,    52,  2673,    11,  2358,
            199,     4,   286,  2358,   954,    52,  3278,    84,  2375,   629,
            731,     7,    28,  2219,   564,     4,   245,  2153,   166,  1057,
           2358,   954, 15326,  1107,   228,   458,     7,  1733,  2219,   155,
              4,   134,   207,     7,    68,   698,     4,  3933,     4,  1541,
           1107,   228,   458,  3824,  1171,    84,   563,     7,  2851, 16707,
           2219,    68,   245,   325,     9,  3973,   327,   148,     5,    76,
              4,     2,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,   

In [ ]:
model(training_set.__getitem__(0)[0])

SequenceClassifierOutput([('logits',
                           tensor([[-0.3340, -0.0336]], grad_fn=<AddmmBackward0>))])

In [ ]:
testing_set = Intents(test_data)
print(testing_set.__getitem__(0)[0].shape)

torch.Size([1, 512])


In [ ]:
training_loader = DataLoader(training_set, batch_size = 2, shuffle = True, drop_last = False)
testing_loader = DataLoader(testing_set, batch_size = 2, shuffle = False, drop_last = False) #不超過16(4-16)

In [ ]:
#BCEWithLogitsLoss 
loss_function = nn.BCEWithLogitsLoss() #CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [ ]:
inp = training_set.__getitem__(0)[0]
output = model(inp)[0]
print(output.shape)

torch.Size([1, 2])


In [ ]:
model = model.cuda()

In [ ]:
n = 0
for i, (sent, label) in enumerate(training_loader):
  total =0
  if n<1: 
    # a = sent.squeeze(1)
    # print(a)
    # print(a.shape)
    # pre = nn.Sigmoid()(a)
    # print('pre',pre)
    # print('pre_shape:',pre.shape)
    print('label:',label)
    print('label_shape:',label.shape)
    print(label.size())
    total += label.size(0)
    print(total)


  else:
    break

  n+=1

label: tensor([0, 0])
label_shape: torch.Size([2])


IndexError: ignored

In [ ]:
max_epochs = 3
model = model.train()
loss_function = nn.BCELoss()
m = nn.Sigmoid()

for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        # print('sent: ',sent)
        # print('Label:', label)
        optimizer.zero_grad()
        sent = sent.squeeze(1)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
          output = model.forward(sent)[0]
          _, predicted = torch.max(output, 1)
          # print('output:', output.shape)
          # print('label:', label)
          label = F.one_hot(label, num_classes = 2)
          loss = loss_function(m(output), label.type(torch.float32))
          loss.backward()
          optimizer.step()
        
        if i%100 == 0:
            TP = 0
            TN = 0
            FP = 0
            FN = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(1)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)

                # TP    predict 和 label 同時爲1
                TP += ((predicted == 1) & (label == 1)).cpu().sum()
                # TN    predict 和 label 同時爲0
                TN += ((predicted == 0) & (label == 0)).cpu().sum()
                # FN    predict 0 label 1
                FN += ((predicted == 0) & (label == 1)).cpu().sum()
                # FP    predict 1 label 0
                FP += ((predicted == 1) & (label == 0)).cpu().sum()

                # correct += (predicted.cpu() == label.cpu()).sum()
            # accuracy = 100.00 * correct.numpy() / total
            p = TP / (TP + FP)
            r = TP / (TP + FN)
            F1 = 2 * r * p / (r + p)
            acc = (TP + TN) / (TP + TN + FP + FN)
            print('Iteration: {}. Loss: {}. Accuracy: {}%. TP:{}. TN: {}. FP: {}. FN: {}' .format(i, loss.item(), acc, TP, TN, FP, FN))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH -- 0
Iteration: 0. Loss: 0.2345241904258728. Accuracy: 0.8842231631278992%. TP:0. TN: 1474. FP: 0. FN: 193
Iteration: 100. Loss: 0.11905799061059952. Accuracy: 0.8842231631278992%. TP:0. TN: 1474. FP: 0. FN: 193


KeyboardInterrupt: ignored

In [ ]:
def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    input_msg = input_msg.cuda()
  output = model(input_msg)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(label_to_ix.keys())[pred_label]
  return prediction